# Setup Required Libraries
Import matplotlib, numpy, and animation tools required for visualization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

: 

Run Simulation in Numpy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List, Tuple

@dataclass
class ReactorParams:
    """Physical parameters for the reactor simulation"""
    tube_length: float = 2.0  # meters
    tube_radius: float = 0.1  # meters
    deuterium_mass: float = 3.34e-27  # kg
    deuterium_charge: float = 1.6e-19  # Coulombs
    lens_positions: List[float] = None  # positions of magnetic lenses
    lens_strengths: List[float] = None  # magnetic field strengths
    target_position: float = 1.9  # meters from start
    
class MagneticLens:
    """Represents a magnetic lens in the reactor"""
    def __init__(self, position: float, strength: float, aperture: float):
        self.position = position
        self.strength = strength
        self.aperture = aperture
    
    def get_field(self, x: float, r: float) -> Tuple[float, float]:
        """Calculate magnetic field components (Bx, Br) at given position"""
        dx = x - self.position
        field_strength = self.strength * np.exp(-(dx**2 + r**2)/(2*self.aperture**2))
        Bx = field_strength * dx/self.aperture
        Br = field_strength * r/self.aperture
        return Bx, Br

class PlasmaSimulation:
    def __init__(self):
        self.params = ReactorParams()
        
        # Initialize magnetic lenses
        self.lenses = [
            MagneticLens(0.5, 1.0, 0.05),
            MagneticLens(1.0, 1.2, 0.05),
            MagneticLens(1.5, 1.5, 0.05)
        ]
        
        # Initialize beam particles
        self.n_particles = 1000
        self.beam_positions = np.zeros((self.n_particles, 3))  # x,y,z coordinates
        self.beam_velocities = np.zeros((self.n_particles, 3))
        
        # Set initial conditions
        self.beam_positions[:,0] = 0.0  # start at x=0
        self.beam_velocities[:,0] = 1e5  # initial velocity in x direction
        
        # Random initial radial positions
        r = np.random.uniform(0, 0.02, self.n_particles)
        theta = np.random.uniform(0, 2*np.pi, self.n_particles)
        self.beam_positions[:,1] = r * np.cos(theta)
        self.beam_positions[:,2] = r * np.sin(theta)
        
        self.fusion_events = []

    def get_magnetic_force(self) -> np.ndarray:
        """Calculate magnetic forces on particles"""
        forces = np.zeros_like(self.beam_positions)
        
        for lens in self.lenses:
            for i in range(self.n_particles):
                x = self.beam_positions[i,0]
                r = np.sqrt(self.beam_positions[i,1]**2 + self.beam_positions[i,2]**2)
                Bx, Br = lens.get_field(x, r)
                
                # F = q(v × B)
                v = self.beam_velocities[i]
                B = np.array([Bx, Br*self.beam_positions[i,1]/r, Br*self.beam_positions[i,2]/r])
                F = self.params.deuterium_charge * np.cross(v, B)
                forces[i] += F
                
        return forces

    def check_fusion(self) -> int:
        """Check for fusion events when particles hit target"""
        fusion_count = 0
        for i in range(self.n_particles):
            if self.beam_positions[i,0] >= self.params.target_position:
                fusion_count += 1
                # Reset particle to start
                self.beam_positions[i,0] = 0.0
                r = np.random.uniform(0, 0.02)
                theta = np.random.uniform(0, 2*np.pi)
                self.beam_positions[i,1] = r * np.cos(theta)
                self.beam_positions[i,2] = r * np.sin(theta)
                
        return fusion_count

    def step(self, dt=1e-9):
        """Advance simulation by one timestep"""
        forces = self.get_magnetic_force()
        self.beam_velocities += forces * dt / self.params.deuterium_mass
        self.beam_positions += self.beam_velocities * dt
        
        fusions = self.check_fusion()
        self.fusion_events.append(fusions)
        return fusions

    def run(self, steps=1000):
        """Run simulation for specified number of steps"""
        for _ in range(steps):
            self.step()
        return np.array(self.fusion_events)

# Run simulation and plot results
sim = PlasmaSimulation()
fusion_history = sim.run()

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.plot(fusion_history, label='Fusion events')
plt.xlabel('Time Step')
plt.ylabel('Fusion Events')
plt.title('Fusion Events over Time')
plt.legend()

# Plot final particle positions
plt.subplot(122)
plt.scatter(sim.beam_positions[:,0], 
           np.sqrt(sim.beam_positions[:,1]**2 + sim.beam_positions[:,2]**2),
           alpha=0.1)
plt.xlabel('Position (m)')
plt.ylabel('Radial Distance (m)')
plt.title('Particle Positions')
plt.tight_layout()
plt.show()

# Create Particle Animation Class
Create a class that inherits from the PlasmaSimulation class and adds animation capabilities to track particle positions over time.

In [ ]:
class PlasmaAnimation(PlasmaSimulation):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fig, self.ax = plt.subplots()
        self.scat = self.ax.scatter([], [], alpha=0.1)
        self.ax.set_xlim(-1, 1)
        self.ax.set_ylim(-1, 1)
        self.ax.set_xlabel('Position (m)')
        self.ax.set_ylabel('Radial Distance (m)')
        self.ax.set_title('Particle Positions Over Time')

    def init_animation(self):
        # Initialize with actual starting positions
        positions = np.column_stack((
            self.beam_positions[:, 0],  # x coordinates
            np.sqrt(self.beam_positions[:, 1]**2 + self.beam_positions[:, 2]**2)  # radial distances
        ))
        self.scat.set_offsets(positions)
        return self.scat,

    def update_animation(self, frame):
        self.step()
        positions = np.column_stack((self.beam_positions[:, 0], 
                                     np.sqrt(self.beam_positions[:, 1]**2 + self.beam_positions[:, 2]**2)))
        self.scat.set_offsets(positions)
        return self.scat,

    def animate(self, frames=100, interval=50):
        self.anim = FuncAnimation(self.fig, self.update_animation, init_func=self.init_animation,
                                  frames=frames, interval=interval, blit=True)
        plt.show()

# Create an instance of the animation class and run the animation
plasma_anim = PlasmaAnimation()
plasma_anim.animate()

# Implement Animation Frame Update
Define animation update function that handles particle position updates and redraws the plot for each frame.

In [ ]:
def update_animation(self, frame):
    """Update function for animation"""
    self.step()  # Advance the simulation by one step
    # Update particle positions
    positions = np.column_stack((self.beam_positions[:, 0], 
                                 np.sqrt(self.beam_positions[:, 1]**2 + self.beam_positions[:, 2]**2)))
    self.scat.set_offsets(positions)  # Update scatter plot with new positions
    return self.scat,

# Display 3D Animation
Create a 3D visualization showing particle trajectories and interactions in three-dimensional space using plt.figure(figsize=(10,10), projection='3d').

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

class PlasmaAnimation3D(PlasmaSimulation):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fig = plt.figure(figsize=(10, 10))
        self.ax = self.fig.add_subplot(111, projection='3d')
        self.scat = self.ax.scatter([], [], [], alpha=0.1)
        self.ax.set_xlim(-1, 1)
        self.ax.set_ylim(-1, 1)
        self.ax.set_zlim(-1, 1)
        self.ax.set_xlabel('X Position (m)')
        self.ax.set_ylabel('Y Position (m)')
        self.ax.set_zlabel('Z Position (m)')
        self.ax.set_title('3D Particle Trajectories Over Time')

    def init_animation(self):
        # Initialize with actual starting 3D positions
        self.scat._offsets3d = (
            self.beam_positions[:, 0],  # x coordinates
            self.beam_positions[:, 1],  # y coordinates
            self.beam_positions[:, 2]   # z coordinates
        )
        return self.scat,

    def update_animation(self, frame):
        self.step()
        self.scat._offsets3d = (self.beam_positions[:, 0], 
                                self.beam_positions[:, 1], 
                                self.beam_positions[:, 2])
        return self.scat,

    def animate(self, frames=100, interval=50):
        self.anim = FuncAnimation(self.fig, self.update_animation, init_func=self.init_animation,
                                  frames=frames, interval=interval, blit=True)
        plt.show()

# Create an instance of the 3D animation class and run the animation
plasma_anim_3d = PlasmaAnimation3D()
plasma_anim_3d.animate()

# Create 2D Projection Animation
Implement a 2D projection view of the particle motion showing beam spread and interactions from different viewing angles.

In [ ]:
# Create 2D Projection Animation
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

class PlasmaAnimation(PlasmaSimulation):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fig, self.ax = plt.subplots()
        self.scat = self.ax.scatter([], [], alpha=0.1)
        self.ax.set_xlim(-1, 1)
        self.ax.set_ylim(-1, 1)
        self.ax.set_xlabel('Position (m)')
        self.ax.set_ylabel('Radial Distance (m)')
        self.ax.set_title('Particle Positions Over Time')

    def init_animation(self):
        # Initialize with actual starting positions
        positions = np.column_stack((
            self.beam_positions[:, 0],  # x coordinates
            np.sqrt(self.beam_positions[:, 1]**2 + self.beam_positions[:, 2]**2)  # radial distances
        ))
        self.scat.set_offsets(positions)
        return self.scat,

    def update_animation(self, frame):
        self.step()
        positions = np.column_stack((self.beam_positions[:, 0], 
                                     np.sqrt(self.beam_positions[:, 1]**2 + self.beam_positions[:, 2]**2)))
        self.scat.set_offsets(positions)
        return self.scat,

    def animate(self, frames=100, interval=50):
        self.anim = FuncAnimation(self.fig, self.update_animation, init_func=self.init_animation,
                                  frames=frames, interval=interval, blit=True)
        plt.show()

# Create an instance of the animation class and run the animation
plasma_anim = PlasmaAnimation()
plasma_anim.animate()

## Beam-line Visualization

In [ ]:
class BeamLineAnimation(PlasmaSimulation):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fig, self.ax = plt.subplots(figsize=(12, 4))
        
        # Setup reactor tube visualization
        self.tube_top = self.ax.axhline(y=self.params.tube_radius, color='gray', linestyle='--')
        self.tube_bottom = self.ax.axhline(y=-self.params.tube_radius, color='gray', linestyle='--')
        
        # Setup magnetic lens indicators
        for lens in self.lenses:
            self.ax.axvline(x=lens.position, color='blue', alpha=0.3, linewidth=5)
        
        # Setup target indicator
        self.ax.axvline(x=self.params.target_position, color='red', alpha=0.3, linewidth=5)
        
        # Setup particle scatter plot
        self.scat = self.ax.scatter([], [], c=[], cmap='plasma', alpha=0.6)
        
        # Set plot limits and labels
        self.ax.set_xlim(0, self.params.tube_length)
        self.ax.set_ylim(-1.5*self.params.tube_radius, 1.5*self.params.tube_radius)
        self.ax.set_xlabel('Beam Direction (m)')
        self.ax.set_ylabel('Radial Position (m)')
        self.ax.set_title('Plasma Beam Path')

    def init_animation(self):
        # Initialize with actual positions and velocities
        positions = np.column_stack((
            self.beam_positions[:, 0],  # x position
            self.beam_positions[:, 1]   # y position (using only one transverse coordinate)
        ))
        self.scat.set_offsets(positions)
        
        # Color based on velocity magnitude
        velocities = np.linalg.norm(self.beam_velocities, axis=1)
        self.scat.set_array(velocities)
        return self.scat,

    def update_animation(self, frame):
        self.step()
        positions = np.column_stack((
            self.beam_positions[:, 0], 
            self.beam_positions[:, 1]
        ))
        self.scat.set_offsets(positions)
        
        velocities = np.linalg.norm(self.beam_velocities, axis=1)
        self.scat.set_array(velocities)
        return self.scat,

    def animate(self, frames=100, interval=50):
        self.anim = FuncAnimation(self.fig, self.update_animation, 
                                init_func=self.init_animation,
                                frames=frames, interval=interval, blit=True)
        plt.show()

# Create and run animation
beam_anim = BeamLineAnimation()
beam_anim.animate()

In [ ]:
class BeamInstabilityAnimation(PlasmaSimulation):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fig, self.ax = plt.subplots(figsize=(15, 5))
        
        # Magnetic lens parameters
        self.lens_position = 0.5
        self.lens_strength = 2.0
        self.kink_frequency = 0.5
        self.kink_amplitude = 0.02
        
        # Setup visualization
        self.tube_boundary = self.params.tube_radius
        self.ax.axhline(y=self.tube_boundary, color='gray', linestyle='--', alpha=0.5)
        self.ax.axhline(y=-self.tube_boundary, color='gray', linestyle='--', alpha=0.5)
        
        # Magnetic lens visualization
        lens_width = 0.1
        lens_patch = plt.Rectangle((self.lens_position - lens_width/2, -self.tube_boundary*1.2),
                                 lens_width, 2.4*self.tube_boundary,
                                 color='blue', alpha=0.2)
        self.ax.add_patch(lens_patch)
        
        # Target plate
        self.ax.axvline(x=self.params.target_position, color='red', linewidth=2, alpha=0.5)
        
        # Particle scatter plot
        self.scat = self.ax.scatter([], [], c='yellow', alpha=0.6, s=1)
        
        # Plot settings
        self.ax.set_xlim(0, self.params.tube_length)
        self.ax.set_ylim(-2*self.tube_boundary, 2*self.tube_boundary)
        self.ax.set_xlabel('Beam Direction (m)')
        self.ax.set_ylabel('Radial Position (m)')
        self.ax.set_title('Plasma Beam with Instabilities')

    def apply_kink_modes(self, positions, time):
        """Apply kink instability perturbations"""
        x = positions[:, 0]
        # Multiple mode perturbations
        kink = (self.kink_amplitude * np.sin(2*np.pi*self.kink_frequency*x + time) + 
                0.5*self.kink_amplitude * np.sin(4*np.pi*self.kink_frequency*x + 2*time))
        
        # Apply perturbation after magnetic lens
        mask = x > self.lens_position
        positions[mask, 1] += kink[mask]
        return positions

    def init_animation(self):
        positions = np.column_stack((
            self.beam_positions[:, 0],
            self.beam_positions[:, 1]
        ))
        positions = self.apply_kink_modes(positions, 0)
        self.scat.set_offsets(positions)
        return self.scat,

    def update_animation(self, frame):
        self.step()
        positions = np.column_stack((
            self.beam_positions[:, 0],
            self.beam_positions[:, 1]
        ))
        
        # Apply magnetic focusing near lens
        dist_to_lens = np.abs(positions[:, 0] - self.lens_position)
        lens_effect = np.exp(-dist_to_lens**2 / 0.1**2)
        positions[:, 1] *= (1 - 0.5*lens_effect)
        
        # Apply kink modes
        positions = self.apply_kink_modes(positions, frame/10)
        
        # Update particle positions
        self.scat.set_offsets(positions)
        return self.scat,

    def animate(self, frames=200, interval=50):
        self.anim = FuncAnimation(self.fig, self.update_animation,
                                init_func=self.init_animation,
                                frames=frames, interval=interval, blit=True)
        plt.show()

# Run animation
beam_instability = BeamInstabilityAnimation()
beam_instability.animate()

In [ ]:
class KinkModeAnimation(PlasmaSimulation):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # Kink mode parameters
        self.safety_factor = 2.0  # q > 1 for stability
        self.growth_rate = 0.1    # Growth rate of instability
        self.helical_mode = 2     # m=2 kink mode
        self.B_theta = 0.5        # Poloidal field strength
        self.B_z = 2.0           # Axial field strength
        
        # Setup visualization
        self.fig = plt.figure(figsize=(15, 5))
        self.ax = self.fig.add_subplot(111)
        
        # Plasma column visualization
        self.tube_radius = self.params.tube_radius
        self.column = self.ax.plot([], [], 'b-', lw=2, alpha=0.5)[0]
        
        # Particle scatter plot
        self.scat = self.ax.scatter([], [], c='yellow', alpha=0.6, s=2)
        
        # Plot settings
        self.ax.set_xlim(0, self.params.tube_length)
        self.ax.set_ylim(-2*self.tube_radius, 2*self.tube_radius)
        self.ax.set_xlabel('z (m)')
        self.ax.set_ylabel('r (m)')
        self.ax.set_title('Plasma Column with Kink Mode')

    def calculate_kink_displacement(self, z, t):
        """Calculate helical displacement due to kink instability"""
        # Basic helical displacement
        k_z = self.helical_mode * 2*np.pi / self.params.tube_length
        omega = np.sqrt(self.B_theta * k_z / (self.params.deuterium_mass * self.B_z))
        
        # Growth term from magnetic pressure imbalance
        growth = np.exp(self.growth_rate * t)
        
        # Safety factor correction
        q_factor = 1.0 / (1.0 - 1.0/self.safety_factor**2)
        
        # Helical displacement
        x = 0.1 * self.tube_radius * growth * np.sin(k_z*z - omega*t) * q_factor
        y = 0.1 * self.tube_radius * growth * np.cos(k_z*z - omega*t) * q_factor
        
        return x, y

    def init_animation(self):
        # Initialize plasma column
        z = np.linspace(0, self.params.tube_length, 100)
        x, y = self.calculate_kink_displacement(z, 0)
        self.column.set_data(z, x)
        
        # Initialize particles
        positions = np.column_stack((
            self.beam_positions[:, 0],
            self.beam_positions[:, 1]
        ))
        self.scat.set_offsets(positions)
        return self.column, self.scat

    def update_animation(self, frame):
        t = frame * 0.01  # Time scaling
        
        # Update plasma column
        z = np.linspace(0, self.params.tube_length, 100)
        x, y = self.calculate_kink_displacement(z, t)
        self.column.set_data(z, x)
        
        # Update particles
        self.step()
        z_pos = self.beam_positions[:, 0]
        x_displacement, y_displacement = self.calculate_kink_displacement(z_pos, t)
        
        # Add kink displacement to particle positions
        positions = np.column_stack((
            z_pos,
            self.beam_positions[:, 1] + x_displacement
        ))
        
        # Add magnetic pressure effects
        r = np.sqrt(positions[:, 1]**2)
        pressure_factor = 1 + 0.2 * np.sin(2*np.pi*z_pos/self.params.tube_length)
        positions[:, 1] *= pressure_factor
        
        self.scat.set_offsets(positions)
        return self.column, self.scat

    def animate(self, frames=200, interval=50):
        self.anim = FuncAnimation(self.fig, self.update_animation,
                                init_func=self.init_animation,
                                frames=frames, interval=interval, blit=True)
        plt.show()

# Run animation
kink_mode = KinkModeAnimation()
kink_mode.animate()

In [ ]:
class KinkModeAnimation(PlasmaSimulation):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # Physical parameters
        self.safety_factor = 2.0
        self.growth_rate = 0.1
        self.current_density = 1e6  # A/m^2
        self.B0 = 2.0  # Tesla
        
        # Setup visualization
        self.fig = plt.figure(figsize=(15, 5))
        self.ax = self.fig.add_subplot(111)
        self.column = self.ax.plot([], [], 'b-', lw=2, alpha=0.5)[0]
        self.scat = self.ax.scatter([], [], c='yellow', alpha=0.6, s=2)
        
        self._setup_plot()

    def _setup_plot(self):
        self.ax.set_xlim(0, self.params.tube_length)
        self.ax.set_ylim(-2*self.params.tube_radius, 2*self.params.tube_radius)
        self.ax.set_xlabel('z (m)')
        self.ax.set_ylabel('r (m)')
        self.ax.set_title('Plasma Column with Kink Mode Instability')

    def displacement_tensor(self, z, t, r, theta):
        """Calculate displacement tensor D"""
        # Wave parameters
        k = 2*np.pi/self.params.tube_length  # Wavenumber
        omega = np.sqrt(k * self.B0 / (self.params.deuterium_mass * np.pi))
        
        # Growth factor from linear stability analysis
        gamma = self.growth_rate * np.sqrt(self.current_density / (self.B0 * self.safety_factor))
        growth = np.exp(gamma * t)
        
        # Helical displacement components
        D_x = growth * r * np.cos(k*z - omega*t + theta)
        D_y = growth * r * np.sin(k*z - omega*t + theta)
        
        return np.array([D_x, D_y, np.zeros_like(D_x)])

    def current_density_tensor(self, r, theta, z, t):
        """Calculate current density tensor J"""
        # Gaussian radial profile
        J_z = self.current_density * np.exp(-r**2/(2*self.params.tube_radius**2))
        # Add perturbation from kink mode
        perturbation = 0.1 * np.sin(theta) * np.cos(2*np.pi*z/self.params.tube_length - t)
        J_z *= (1 + perturbation)
        
        return np.array([np.zeros_like(J_z), np.zeros_like(J_z), J_z])

    def calculate_positions(self, z, t):
        """Calculate particle positions including kink mode effects"""
        r = np.sqrt(self.beam_positions[:,1]**2 + self.beam_positions[:,2]**2)
        theta = np.arctan2(self.beam_positions[:,2], self.beam_positions[:,1])
        
        # Calculate displacement and current tensors
        D = self.displacement_tensor(z, t, r, theta)
        J = self.current_density_tensor(r, theta, z, t)
        
        # Calculate magnetic field from current
        B_theta = self.B0 * J[2] / (2*np.pi*r * self.current_density)
        
        # Update positions including kink displacement
        new_positions = np.zeros_like(self.beam_positions)
        new_positions[:,0] = z
        new_positions[:,1] = r * np.cos(theta) + D[0]
        new_positions[:,2] = r * np.sin(theta) + D[1]
        
        return new_positions

    def init_animation(self):
        z = np.linspace(0, self.params.tube_length, 100)
        positions = self.calculate_positions(z, 0)
        
        # Update visualization
        self.column.set_data(positions[:,0], positions[:,1])
        self.scat.set_offsets(np.c_[positions[:,0], 
                                   np.sqrt(positions[:,1]**2 + positions[:,2]**2)])
        return self.column, self.scat

    def update_animation(self, frame):
        t = frame * 0.01
        z = self.beam_positions[:,0]
        positions = self.calculate_positions(z, t)
        
        # Update particle positions
        self.beam_positions = positions
        self.step()
        
        # Update visualization
        self.column.set_data(positions[:,0], positions[:,1])
        self.scat.set_offsets(np.c_[positions[:,0], 
                                   np.sqrt(positions[:,1]**2 + positions[:,2]**2)])
        return self.column, self.scat

    def animate(self, frames=100, interval=50):
      """
      Animate the plasma beam simulation
      
      Parameters:
          frames (int): Number of animation frames
          interval (int): Time between frames in milliseconds
      """
      self.anim = FuncAnimation(self.fig, 
                              self.update_animation, 
                              init_func=self.init_animation,
                              frames=frames, 
                              interval=interval, 
                              blit=True,
                              repeat=False)
      
      # Add colorbar to show particle energies
      velocities = np.linalg.norm(self.beam_velocities, axis=1)
      self.scat.set_array(velocities)
      plt.colorbar(self.scat, label='Particle Velocity (m/s)')
      
      # Adjust plot layout
      plt.tight_layout()
      
      # Show the animation
      plt.show()

# Create instance of animation class
plasma_animation = PlasmaAnimation()

# Call animate method with optional parameters
# frames: number of frames to animate (default 100)
# interval: delay between frames in milliseconds (default 50ms)
plasma_animation.animate(frames=200, interval=20)